## Import libraries

In [1]:
import os
import json
import spotipy
import requests
import numpy as np
import pandas as pd
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity

### I will read in the dataframe I have build my model on, and the one I use for grouping by tempo, and while I'm at it I will read in the dataframe of the names and the uris we had created in the modeling process so we can Identify the songs 
***doing so I will not have to tinker with the api and it's a HUGE detour***

### I will set all of the indexes as col=index

In [2]:
matrix = pd.read_csv('./Data/recommender.csv')
df= pd.read_csv('./Data/identifier.csv')
df_id= pd.read_csv('./Data/uri.csv')

In [3]:
df_id.set_index('index',inplace=True)
df_id.head(1)

,uri
index,
"siren 042, lala lala",spotify:track:75nN4kH1uzSsUdMrdUVfrq


In [4]:
df.set_index('index',inplace=True)
df.head(1)

,tempo,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,time_signature,duration_ms
index,,,,,,,,,,,,,
"siren 042, lala lala",-0.50495,0.45834,-0.112834,1.722145,0.482366,0.570691,-0.546041,1.386794,-0.389266,-0.545219,-0.481937,0.231248,-0.882392


In [5]:
matrix.set_index('index',inplace=True)

In [9]:
matrix.tail(20)

,"siren 042, lala lala","make me a song, eleanor friedberger","uprising, muse","time is running out, muse","knights of cydonia, muse","vicious, lou reed","like a hurricane - 2017 remaster, neil young","smile like you mean it, the killers","season of the witch, donovan","harvest moon, neil young",...,"riders on the storm, the doors","you never can tell, chuck berry","you really got me - mono mix, the kinks","whole lotta love - 1990 remaster, led zeppelin","water under the bridge, adele","mannish boy, muddy waters","white rabbit, jefferson airplane","people are strange, the doors","valerie - live at bbc radio 1 live lounge, london / 2007, amy winehouse","fortunate son, creedence clearwater revival"
index,,,,,,,,,,,,,,,,,,,,,
"the chain - 2004 remaster, fleetwood mac",0.162869,0.302162,0.354614,0.407761,0.455153,-0.058197,0.311060,0.019802,0.004386,-0.157986,...,0.227509,0.026110,0.365219,0.026712,0.052641,-0.433093,-0.210926,-0.567551,-0.425536,-0.081195
"old time rock & roll, bob seger",0.225887,0.336767,0.467693,0.306508,-0.026345,0.329105,0.103032,0.100535,-0.000060,-0.410708,...,-0.025929,0.350673,0.834715,-0.152212,0.265448,-0.189588,-0.431105,0.061155,0.120656,0.492856
"little wing, jimi hendrix",0.408986,-0.144852,0.049715,0.146295,-0.029963,-0.080615,-0.212448,0.055243,0.306720,0.079831,...,-0.194693,-0.170776,0.138131,0.182296,0.282934,-0.309487,0.462743,-0.125278,0.069438,0.108930
"johnny b. goode, chuck berry",0.369110,-0.382183,-0.298397,-0.155452,-0.050617,-0.065642,-0.213153,-0.106872,-0.541278,-0.381846,...,-0.080415,0.834493,0.525690,0.090850,-0.410164,0.079736,-0.332469,0.250629,-0.045561,-0.025912
"me and bobby mcgee, janis joplin",-0.185271,0.193643,-0.160175,-0.523584,-0.445964,-0.142879,-0.139156,-0.320774,0.866439,0.560743,...,-0.054946,-0.357408,-0.395412,-0.080465,-0.217294,-0.008237,0.522939,-0.173784,-0.014906,0.086028
"you know i'm no good, amy winehouse",0.332348,0.421858,0.498185,0.512089,0.187517,0.185403,0.367378,0.053902,0.121335,-0.290142,...,0.254499,-0.016731,0.633587,-0.072336,0.412122,-0.254584,-0.390847,-0.176951,0.102453,0.163012
"work song, hozier",0.788561,-0.280786,-0.298291,-0.065633,0.057457,-0.478525,-0.138681,-0.276297,-0.221412,0.381385,...,-0.016539,0.090098,-0.206176,-0.010102,-0.291562,-0.230132,0.433931,0.068460,0.301069,-0.358099
"sunny afternoon - mono mix, the kinks",-0.145946,0.559552,0.453947,-0.165293,-0.346160,0.065952,-0.264636,-0.046213,0.544584,0.424690,...,-0.518920,-0.357501,-0.065394,-0.427638,0.014418,-0.292551,0.531003,-0.124358,-0.006136,0.748702
"friend of the devil, grateful dead",0.047364,-0.087774,-0.409169,-0.320054,-0.420847,-0.115503,-0.251683,-0.352999,-0.126810,-0.043805,...,-0.005583,0.166836,0.004976,0.063008,-0.280223,0.511027,-0.112247,0.326650,0.025999,-0.017961


### I will once again request a token from Spoitify, this is just like the first time we requested a token, but this time we need to change the scoope to ***playlist-modify-private*** 
***make sure you set your token to a global variable so you can use it in the next step***

In [7]:
def get_token(scope=None):
    
    redirect_uri = os.environ['SPOTIPY_REDIRECT_URI']
    username = os.environ['SPOTIPY_USERNAME']
    client_id = os.environ['SPOTIPY_CLIENT_ID']
    client_secret = os.environ['SPOTIPY_CLIENT_SECRET']
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
    return token

token = get_token(scope='playlist-modify-private')
sp = spotipy.Spotify(auth=token)

In [8]:
get_token()

'BQCas8VG4E2pTzLryrLI21lncRYVl6A2tcI1dEfeNm1NcdKtxa5QFxQVKLIyihh_3sR9E4gHs0MJZRT0TlQHXKAUuKKSizDwd1JHU-YebT0435KjZbCJ6Pym83TtyB087Kh8q_fSCoDAxmCOtFDSvktY9Cx8gokFkHzPJNYYymIYCnj1FIU-GUIx0hmOi92OllUl9r4uAVDPI_faX3GFc0D3'

### now I will run the function where I can get rcomendations based on a song, I will set my output to a set of uri's
***(set of uri's of the songs that are recommended based on the song of choice)***
### make sure you set your output to a global variable so we can use it to dump tracks in the playlist!

In [19]:
def play_list(choice= None):
    list_of_recoms =[]
    uri= []
    for song in df.loc[df.index.str.contains(choice)].sort_values(by='tempo', ascending = True).index:
        list_of_recoms.append(matrix[song].sort_values(ascending = False)[0:10])
        list_of_recoms.append(matrix[song].sort_values(ascending = False)[19:9:-1])        
        recom= pd.DataFrame(list_of_recoms).T
        recom= recom.drop(columns=choice)
        uri_df = pd.merge(recom,df_id, left_index=True, right_index=True)
        uris= uri_df.reset_index(drop=True)
        uris= uri_df.set_index('uri')
        uri.append(uris.index)
        return uri[0].values
    
    
uri_request= play_list(choice='whole lotta love - 1990 remaster, led zeppelin')

### now using the token we had recived from spotify we will make a playlist, we will recive a playlist Id which we need to set to a global variable, so we can access the playlist and dump music in it!

In [20]:
def create_playlist(song= None):
    endpoint_url = f"https://api.spotify.com/v1/users/{os.environ['SPOTIPY_USERNAME']}/playlists"

    request_body = json.dumps({"name": f"playlist based on {song}",
                                "description": f"play list based on {song}",
                                "public": False })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})

    url = response.json()['external_urls']['spotify']
    playlist_id = response.json()['id']
    print(f'PLAYLIST RESPONSE STATUS: {response.status_code}')
    print(f'PLAYLIST ID: {playlist_id}')
    print(f'URL: {url}')
    return playlist_id
playlist_id_returned= create_playlist('whole lotta love - 1990 remaster, led zeppelin')

PLAYLIST RESPONSE STATUS: 201
PLAYLIST ID: 72ebzXXJ9ZB6ZvQWf3fQ9X
URL: https://open.spotify.com/playlist/72ebzXXJ9ZB6ZvQWf3fQ9X


### lastly using the potipy library, we need to call on our username, the playlist_id and the list of recomended uri's that our recomender had build to dump music in our playlist! 

In [21]:
sp.user_playlist_add_tracks(user=os.environ['SPOTIPY_USERNAME'], playlist_id=playlist_id_returned, tracks=uri_request)

{'snapshot_id': 'MywxODExYWM4ZjVhM2VhODIxYmViOTEzZjAzYjhjOGFmMjQwMjZjODc5'}

# et voila!